In [1]:
# import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install python-dotenv --upgrade
# !{sys.executable} -m pip install pandas --upgrade
# !{sys.executable} -m pip install gql[requests]

## Imports

In [2]:
# For caching
from functools import lru_cache

# To read environment property file
import os
from dotenv import load_dotenv
from pathlib import Path

# GQL clinet libraries
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport

# Date calculations
from datetime import datetime, timedelta
import time

# For dataframe
import pandas as pd

## Load environment variables

In [3]:
dotenv_path = Path('.env/graph')
load_dotenv(dotenv_path=dotenv_path)
GRAPH_API_KEY = os.getenv('GRAPH_API_KEY')

## Constants

In [4]:
# The graph.com endpoint for ENS
GRAPHQL_ENDPOINT = 'https://gateway.thegraph.com/api/{api_key}/\
subgraphs/id/5XqPmWe6gjyrJtFn9cLy237i4cWw2j9HcUJEXsP5qGtH'.format(api_key=GRAPH_API_KEY)

# DF columns
DF_COLUMNS = ['Name', 'Registration Date', 'Expiry Date', 'Grace Expiry', 'Role - Owner',
              'Role - Manager', 'Role - ETH', 'Resolver', 'Registration Cost', 'NFT', 'Metadata']

# Format for strftime function
TIME_FORMAT = '%-d %b %Y %H:%M:%S %Z'

# NFT URL prefix
NFT_URL_PREFIX = 'https://etherscan.io/nft/0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85/{token_id}'

# Metadata prefix
METADATA_PREFIX = 'https://metadata.ens.domains/mainnet/0x57f1887a8BF19b14fC0dF6Fd9B2acc9Af147eA85/{token_id}'

# Etherscan prefix for an address or a transaction; obj_type could be 'address' or 'tx'
ETHER_URL_PREFIX = 'https://etherscan.io/{obj_type}/{obj_id}'

## Initialize GraphQL client

In [5]:
# Create a transport object
_transport = RequestsHTTPTransport(url=GRAPHQL_ENDPOINT, verify=True, retries=3)
# Create a GraphQL client
client = Client(transport=_transport, fetch_schema_from_transport=True)

## Query to get domain by name

In [6]:
@lru_cache
def get_domain_by_name(name:str):
    query = gql('''
        query Domain ($name: String){
          domains(where: {name: $name}) {
            expiryDate
            registration {
              id
              cost
              expiryDate
              registrationDate
            }
            owner {
              id
            }
            registrant {
              id
            }
            resolvedAddress {
              id
            }
            resolver {
              address
            }
          }
        }
    ''')
    # Execute the query with variables
    params = {'name':name}
    return client.execute(query, variable_values=params)

## Utlity method to create a link

In [7]:
def create_ether_lnk(obj_type:str, obj_id:str)->str:
    # Check the object type
    assert obj_type in ['address', 'tx'], 'Unknown object type'
    # Addd address or tx based on the object type
    url = ETHER_URL_PREFIX.format(obj_type=obj_type, obj_id=obj_id)
    link_name = '{part1} ... {part2}'.format(part1=obj_id[:5], part2=obj_id[-5:])
    return f'<a target="_blank" href="{url}">{link_name}</a>'

In [8]:
def create_domain_row(domain:dict) -> dict:
    # Initialize with keys
    row = dict([(x, None) for x in DF_COLUMNS])

    # Domain name
    x = 0
    if 'name' in domain:
        row[DF_COLUMNS[x]] = domain['name']

    # Registration date
    x += 1
    if 'registration' in domain and 'registrationDate' in domain['registration']:
        row[DF_COLUMNS[x]] = time.strftime(
            TIME_FORMAT, time.gmtime(int(domain['registration']['registrationDate'])))

    # Registration expiry date
    x += 1
    if 'registration' in domain and 'expiryDate' in domain['registration']:
        row[DF_COLUMNS[x]] = time.strftime(
            TIME_FORMAT, time.gmtime(int(domain['registration']['expiryDate'])))
    
    # Domain grace period
    x += 1    
    row[DF_COLUMNS[x]] = time.strftime(TIME_FORMAT, time.gmtime(int(domain['expiryDate'])))

    # The following three (Owner, Manager, ETH) are roles
    # Roles - start -------------------------------------
    # Owner role - who registered the domain (if present)
    x += 1
    if 'registrant' in domain:
        owner_role = domain['registrant']['id']
        row[DF_COLUMNS[x]] = create_ether_lnk(obj_type='address', obj_id=owner_role)

    # Manager role - who manages the domain (if present)
    x += 1
    if 'owner' in domain:
        manager_role = domain['owner']['id']
        row[DF_COLUMNS[x]] = create_ether_lnk(obj_type='address', obj_id=manager_role)

    # ETH role
    x += 1
    if 'resolvedAddress' in domain and domain['resolvedAddress']:
        eth_role = domain['resolvedAddress']['id']
        row[DF_COLUMNS[x]] = create_ether_lnk(obj_type='address', obj_id=eth_role)
    # Roles - end -------------------------------------
    
    # Resolver
    x += 1
    if 'resolver' in domain and domain['resolver']:
        resolver = domain['resolver']['address']
        row[DF_COLUMNS[x]] = create_ether_lnk(obj_type='address', obj_id=resolver)

    # Registration cost
    x += 1
    if 'registration' in domain and 'cost' in domain['registration']:
        # Convert from Wei to decimals
        row[DF_COLUMNS[x]] = int(domain['registration']['cost'])/pow(10, 18)

    # NFT link
    x += 1
    if 'registration' in domain and 'id' in domain['registration']:
        reg_id = domain['registration']['id']
        url = NFT_URL_PREFIX.format(token_id=int(reg_id, 16))
        link_name = '{part1} ... {part2}'.format(part1=reg_id[:5], part2=reg_id[-5:])
        row[DF_COLUMNS[x]] = f'<a target="_blank" href="{url}">{link_name}</a>'

        # Metadata link
        x += 1
        url = METADATA_PREFIX.format(token_id=int(reg_id, 16))
        link_name = '{part1} ... {part2}'.format(part1=reg_id[:5], part2=reg_id[-5:])
        row[DF_COLUMNS[x]] = f'<a target="_blank" href="{url}">{link_name}</a>'

    return row

In [9]:
def display_domain(name:str, domain:dict) -> None:
    # Get the domain as a row
    row = create_domain_row(domain=domain)
    # Remove keys with None values
    row = {k: v for k, v in row.items() if v is not None}
    
    # Create a DF using keys as columns
    df = pd.DataFrame(columns = row.keys())
    # Append the row
    df.loc[len(df)] = row
    
    # Apply styles
    return df.style.\
        hide(axis='index').\
        set_caption(name).\
        set_properties(**{'border': '0.1px solid black'}).\
        set_table_styles([
            {'selector': 'th.col_heading', 'props': 'text-align: center'},
            {'selector': 'caption', 'props': [('text-align', 'center'), ('font-size', '14pt')]}]).\
        format({'Registration Cost': '{:,f} ETH'})

## Display domain details

In [10]:
domain_name = 'vitalik.eth'
domain = get_domain_by_name(name=domain_name)['domains'][0]
display_domain(name=domain_name, domain=domain)

Registration Date,Expiry Date,Grace Expiry,Role - Owner,Role - Manager,Role - ETH,Resolver,Registration Cost,NFT,Metadata
6 Feb 2020 18:23:40 GMT,28 Dec 2045 13:25:30 GMT,28 Mar 2046 13:25:30 GMT,0x220 ... a3a9d,0xd8d ... 96045,0xd8d ... 96045,0x231 ... e8e63,0.004041 ETH,0xaf2 ... 103cc,0xaf2 ... 103cc


## Query to get domains for the owner

In [11]:
@lru_cache
def get_domains_by_owner(owner:str) -> dict:
    query = gql('''
        query Domain ($owner: String) {
          domains(
            where: {registration_: {cost_gt: "0"}, registrant: $owner}
            first: 20
            orderBy: registration__registrationDate
            orderDirection: desc
          ) {
            name
            expiryDate
            registration {
              cost
              expiryDate
              registrationDate
              id
            }
            owner {
              id
            }
            resolvedAddress {
              id
            }
            resolver {
              address
            }
          }
        }
    ''')
    # Execute the query with variables
    params = {'owner':owner}
    return client.execute(query, variable_values=params)

## Display owner domain

In [12]:
def display_owner_domains(owner:str, domains:dict) -> pd.DataFrame.style:
    # Collection for domains
    data = []
    for dom in domains:
        row = create_domain_row(dom)
        # Remove keys with None values and append to the list
        data.append({k: v for k, v in row.items() if v is not None})
                
    # Create a DF using keys as columns
    df = pd.DataFrame.from_dict(data)

    # Replace nan values
    df['Resolver'] = df['Resolver'].fillna('---')
    df['Role - ETH'] = df['Role - ETH'].fillna('---')

    # Apply styles
    return df.style.\
        hide(axis='index').\
        set_caption(owner).\
        set_properties(subset=['Resolver','Role - ETH'], **{'text-align': 'center'}).\
        set_properties(**{'border': '0.1px solid black'}).\
        set_table_styles([
            {'selector': 'th.col_heading', 'props': 'text-align: center'},
            {'selector': 'caption', 'props': [('text-align', 'center'), ('font-size', '14pt')]}]).\
        format({'Registration Cost': '{:,f} ETH'})

## Display domains for an owner

In [13]:
owner = '0x220866b1a2219f40e72f5c628b65d54268ca3a9d'
# Get other domains for the owner
owner_domains = get_domains_by_owner(owner=owner)['domains']
display_owner_domains(owner=owner, domains=owner_domains)

Name,Registration Date,Expiry Date,Grace Expiry,Role - Manager,Role - ETH,Resolver,Registration Cost,NFT,Metadata
💆🏽‍♂.eth,8 Aug 2024 08:06:23 GMT,22 Sep 2024 08:06:22 GMT,21 Dec 2024 08:06:22 GMT,0xd64 ... e81be,0x176 ... 49b1e,0x497 ... aba41,0.002528 ETH,0xaec ... 7b18a,0xaec ... 7b18a
$tmnt.eth,30 Dec 2023 21:20:35 GMT,30 Dec 2024 03:09:47 GMT,30 Mar 2025 03:09:47 GMT,0xd8d ... 96045,0xd8d ... 96045,0x231 ... e8e63,0.002175 ETH,0x2d9 ... d13ee,0x2d9 ... d13ee
iamsub.eth,19 Jun 2022 04:05:25 GMT,18 Jun 2025 21:33:01 GMT,16 Sep 2025 21:33:01 GMT,0x220 ... a3a9d,0x22e ... 6edd0,0x497 ... aba41,0.015750 ETH,0x276 ... 2cab6,0x276 ... 2cab6
vitalik.eth,6 Feb 2020 18:23:40 GMT,28 Dec 2045 13:25:30 GMT,28 Mar 2046 13:25:30 GMT,0xd8d ... 96045,0xd8d ... 96045,0x231 ... e8e63,0.004041 ETH,0xaf2 ... 103cc,0xaf2 ... 103cc


## Query to get recent domains

In [14]:
# no point in caching as the start date changes
def get_recent_domains(reg_date:int) -> dict:
    query = gql('''
        query Domain ($reg_date: BigInt) {
          domains(
            where: {registration_: {registrationDate_gte: $reg_date}}
            first: 25
            orderDirection: desc
            orderBy: registration__registrationDate
          ) {
            name
            expiryDate
            registration {
              expiryDate
                  registrationDate
                  id
            }
            owner {
              id
            }
            registrant {
              id
            }
            resolvedAddress {
              id
            }
            resolver {
              address
            }            
          }
        }
    ''')
    # Execute the query with variables
    params = {'reg_date':reg_date}
    return client.execute(query, variable_values=params)

## Display recent domain

In [15]:
def display_recent_domains(reg_date:int, domains:dict) -> pd.DataFrame.style:
    # Collection for domains
    data = []
    for dom in domains:
        row = create_domain_row(dom)
        # Remove keys with None values and append to the list
        data.append({k: v for k, v in row.items() if v is not None})
                
    # Create a DF using keys as columns
    df = pd.DataFrame.from_dict(data)
    # Delete rows when name is too long - this is to make the table nice and tidy
    df = df[~(df['Name'].str.len() > 25)]

    # Replace nan values
    df['Resolver'] = df['Resolver'].fillna('---')
    df['Role - ETH'] = df['Role - ETH'].fillna('---')

    # Apply styles
    return df.style.\
        hide(axis='index').\
        set_caption(f'Since {time.strftime(TIME_FORMAT, time.gmtime(reg_date))}').\
        set_properties(subset=['Name'], **{'text-align': 'left'}).\
        set_properties(subset=['Resolver','Role - ETH'], **{'text-align': 'center'}).\
        set_properties(**{'border': '0.1px solid black'}).\
        set_table_styles([
            {'selector': 'th.col_heading', 'props': 'text-align: center'},
            {'selector': 'caption', 'props': [('text-align', 'center'), ('font-size', '14pt')]}])

## Display recent domain registrations

In [16]:
# Start from last 2 hours
reg_date = int((datetime.today() + timedelta(hours=-2)).timestamp())
# Get the domains
recent_domains = get_recent_domains(reg_date=reg_date)['domains']
display_recent_domains(reg_date=reg_date, domains=recent_domains)

Name,Registration Date,Expiry Date,Grace Expiry,Role - Owner,Role - Manager,Role - ETH,Resolver,NFT,Metadata
ukrayina.eth,15 May 2025 14:26:59 GMT,15 May 2035 14:26:59 GMT,13 Aug 2035 14:26:59 GMT,0xa24 ... dc5f2,0xa24 ... dc5f2,0xa24 ... dc5f2,0x231 ... e8e63,0x145 ... 16df2,0x145 ... 16df2
culés.eth,15 May 2025 14:05:35 GMT,15 May 2026 14:05:35 GMT,13 Aug 2026 14:05:35 GMT,0xc74 ... 0af0e,0xc74 ... 0af0e,0xc74 ... 0af0e,0x231 ... e8e63,0x615 ... bbde7,0x615 ... bbde7
basednetdad.eth,15 May 2025 14:02:35 GMT,15 May 2027 14:02:35 GMT,13 Aug 2027 14:02:35 GMT,0xd44 ... 86401,0xd44 ... 86401,---,0x231 ... e8e63,0xa38 ... 0e42e,0xa38 ... 0e42e
gparkdao.eth,15 May 2025 14:00:23 GMT,15 May 2028 14:00:23 GMT,13 Aug 2028 14:00:23 GMT,0xd44 ... 86401,0xd44 ... 86401,0x06a ... 87544,0x231 ... e8e63,0xcfb ... 9ea0a,0xcfb ... 9ea0a
happyrot.eth,15 May 2025 13:54:35 GMT,15 May 2026 13:54:35 GMT,13 Aug 2026 13:54:35 GMT,0xd44 ... 86401,0xd44 ... 86401,0xc92 ... 47ab2,0x231 ... e8e63,0x8d2 ... a1a12,0x8d2 ... a1a12
oneflow.eth,15 May 2025 13:52:11 GMT,15 May 2037 13:52:11 GMT,13 Aug 2037 13:52:11 GMT,0xd44 ... 86401,0xd44 ... 86401,0xefb ... 95093,0x231 ... e8e63,0x332 ... bb0f8,0x332 ... bb0f8
theforgebvi.eth,15 May 2025 13:45:47 GMT,15 May 2027 13:45:47 GMT,13 Aug 2027 13:45:47 GMT,0xd44 ... 86401,0xd44 ... 86401,0x29b ... d2575,0x231 ... e8e63,0x773 ... 7529a,0x773 ... 7529a
vanil.eth,15 May 2025 13:41:23 GMT,15 May 2026 13:41:23 GMT,13 Aug 2026 13:41:23 GMT,0x5c5 ... 5bc2a,0x5c5 ... 5bc2a,0x5c5 ... 5bc2a,0x231 ... e8e63,0xc37 ... cdb3c,0xc37 ... cdb3c
103.eth,15 May 2025 13:07:11 GMT,14 Aug 2025 20:34:29 GMT,12 Nov 2025 20:34:29 GMT,0xd44 ... 86401,0xd44 ... 86401,0x781 ... 6ea54,0x231 ... e8e63,0xf3e ... 9cc53,0xf3e ... 9cc53
bailiff.eth,15 May 2025 13:06:23 GMT,14 Sep 2025 07:02:47 GMT,13 Dec 2025 07:02:47 GMT,0xd44 ... 86401,0xd44 ... 86401,0xb2c ... 8df8c,0x497 ... aba41,0xacf ... f4beb,0xacf ... f4beb


## Query to get registrations for a domain

In [17]:
@lru_cache
def get_domain_registrations(name:str) -> dict:
    query = gql('''
        query Domain ($name: String) {
            nameRegistereds(where: {registration_: {labelName: $name}}) {
                expiryDate
                registration {
                  registrationDate
                  expiryDate
                  id
                }
                registrant {
                  id
                }
                transactionID
              }        
        }
    ''')
    # Execute the query with variables
    params = {'name':name}
    return client.execute(query, variable_values=params)

## Display domain registrations

In [18]:
def display_domain_registrations(name:str, domains:dict) -> pd.DataFrame.style:    
    # Collection for domains
    data = []
    for dom in domains:
        row = create_domain_row(dom)
        # Remove keys with None values and append to the list
        row_dict = {k: v for k, v in row.items() if v is not None}
        # Append the transaction #
        row_dict['Transaction #'] = create_ether_lnk(obj_type='tx', obj_id=dom['transactionID'])
        data.append(row_dict)
                
    # Create a DF using keys as columns
    df = pd.DataFrame.from_dict(data)

    # Apply styles
    return df.style.\
        hide(axis='index').\
        set_caption(name).\
        set_properties(**{'border': '0.1px solid black'}).\
        set_table_styles([
            {'selector': 'th.col_heading', 'props': 'text-align: center'},
            {'selector': 'caption', 'props': [('text-align', 'center'), ('font-size', '14pt')]}])

In [19]:
# Display the general information about the label
label_name = 'esportsplayer'
domain_name = f'{label_name}.eth'
domain = get_domain_by_name(name=domain_name)['domains'][0]
display_domain(name=domain_name, domain=domain)

Registration Date,Expiry Date,Grace Expiry,Role - Owner,Role - Manager,Role - ETH,Resolver,Registration Cost,NFT,Metadata
20 Apr 2024 04:07:35 GMT,31 Jul 2025 15:59:59 GMT,29 Oct 2025 15:59:59 GMT,0xb2b ... 50afe,0xb2b ... 50afe,0x32e ... da2b7,0x497 ... aba41,10.923358 ETH,0x926 ... 3c219,0x926 ... 3c219


## Display the registration for a domain

In [20]:
domain_regs = get_domain_registrations(name=label_name)['nameRegistereds']
key_rmv = {'registration'}
# Remove keys using dictionary comprehension
filtered_regs = [{k: v for k, v in reg.items() if k not in key_rmv} for reg in domain_regs]
display_domain_registrations(name=label_name, domains=filtered_regs)

Grace Expiry,Role - Owner,Transaction #
24 Jun 2023 21:17:26 GMT,0x283 ... eb7f5,0x3e0 ... 42d7d
20 Apr 2025 09:56:47 GMT,0xb2b ... 50afe,0xb97 ... 6557e
4 May 2020 00:00:00 GMT,0x610 ... 5c662,0x092 ... 2c661
